In [1]:
from __future__ import print_function
import json
import numpy as np
from util import *
from skutil import *

from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

from collections import defaultdict
from collections import Counter

from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
with open('../data/yelp_academic_dataset_user.json') as f:
    users = [json.loads(line) for line in f]
with open('../data/yelp_academic_dataset_review.json') as f:
    reviews = [json.loads(line) for line in f]
with open('../data/yelp_academic_dataset_business.json') as f:
    businesses = [json.loads(line) for line in f]

In [3]:
# Split users into train and test set.
usersTrain, usersTest = train_test_split(users, random_state=0)

In [4]:
userReviewMap = mapReviewsByUsers(reviews)
userMap = mapUsers(users)
businessMap = mapBusinesses(businesses)

In [5]:
centroid, label = getKmeans(reviews, businessMap)
localEliteFriends = getLocalEliteFriends(reviews, userMap, label)

C:\Anaconda\lib\site-packages\scipy\cluster\vq.py:600: UserWarning: One of the clusters is empty. Re-run kmean with a different initialization.
  warnings.warn("One of the clusters is empty. "


In [10]:
baseUserFeatures = {'average_stars', 'fans', 'review_count'}

# Seperate pipelines for processing different subsets of features.
base = make_pipeline(SelectKeys(baseUserFeatures), DictVectorizer())
compliments = make_pipeline(SelectValue('compliments'), DictVectorizer())
votes = make_pipeline(SelectValue('votes'), DictVectorizer())
numFriends = make_pipeline(SelectValue('friends'), ApplyFunc(lambda x: len(x)), IdentityVectorizer())
tfidf = make_pipeline(UserReviewText(userReviewMap), TfidfVectorizer(max_features=1000))
votes = make_pipeline(SelectValue('votes'), DictVectorizer())
localElites = make_pipeline(UserEliteFriends(localEliteFriends), IdentityVectorizer())

#Union all the features together
userFeatures = make_union(base, tfidf, localElites)

In [11]:
nbclf = make_pipeline(userFeatures, MultinomialNB())

In [12]:
nbclf.fit(usersTrain, [isElite(u) for u in usersTrain])

Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(steps=[('selectkeys', SelectKeys(keys=set(['fans', 'review_count', 'average_stars']))), ('dictvectorizer', DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=True))])), ...ormer_weights=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [ ]:
predictions = nbclf.predict(usersTest)
print(classification_report([isElite(u) for u in usersTest], predictions))

In [9]:
logregclf = make_pipeline(userFeatures, LogisticRegression())

In [10]:
logregclf.fit(usersTrain, [isElite(u) for u in usersTrain])

Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(steps=[('selectkeys', SelectKeys(keys={'fans', 'average_stars', 'review_count'})), ('dictvectorizer', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True))])), ('pi...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [13]:
predictions = logregclf.predict(usersTest)
print(classification_report([isElite(u) for u in usersTest], predictions))

NameError: name 'logregclf' is not defined

In [12]:
svmclf = make_pipeline(userFeatures, LinearSVC())

In [13]:
svmclf.fit(usersTrain, [isElite(u) for u in usersTrain])

Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(steps=[('selectkeys', SelectKeys(keys={'fans', 'average_stars', 'review_count'})), ('dictvectorizer', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True))])), ('pi...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [14]:
predictions = svmclf.predict(usersTest)
print(classification_report([isElite(u) for u in usersTest], predictions))

             precision    recall  f1-score   support

      False       0.98      0.98      0.98     85414
       True       0.72      0.79      0.75      6265

avg / total       0.97      0.96      0.96     91679

